# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
import requests
import json
import logging
import pandas as pd
import azureml.core
import joblib

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment, ScriptRunConfig
from azureml.train.sklearn import SKLearn


## Workspace

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'HyperDrive-CapstoneProj-experiment-0'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: udc-machinelearning01
Azure region: eastus2
Subscription id: 7023dccf-8c31-41db-809e-9f442490bfe7
Resource group: rg-udc-ml-dev-eastus-01


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
dataset = Dataset.get_by_name(ws, name = 'heart_failure_clinical_records_dataset')
dataset.take(10).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
5,90.0,1,47,0,40,1,204000.00,2.1,132,1,1,8,1
6,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1
7,60.0,1,315,1,60,0,454000.00,1.1,131,1,1,10,1
8,65.0,0,157,0,65,0,263358.03,1.5,138,0,0,10,1
9,80.0,1,123,0,35,1,388000.00,9.4,133,1,1,10,1


## Computer Cluster

In [5]:
# Personal Subcription - using existing cluster
cluster_name = "udc-ml-cluster-02"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new cluster.')
    # 4 cores, 8GB RAM, 32GB storage
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_F4s_v2',
                                                          max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Creating a new cluster.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [20]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval = 2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        "--C" :        choice(0.1,1,2,5,10),
        "--max_iter" : choice(10,100,150,200,250)
    }
)

#TODO: Create hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
src = ScriptRunConfig(
    source_directory= './',
    script="train.py",
    compute_target=cluster,
    environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling = param_sampling, 
    policy = early_termination_policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 20,
    max_concurrent_runs = 4,
    run_config = src
)

In [58]:
#TODO: Submit your experiment
hdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [59]:
RunDetails(hdrive_run).show()
hdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_bac65b68-437e-4e9b-83f8-a42d7476509a',
 'target': 'udc-ml-cluster-02',
 'status': 'Completed',
 'startTimeUtc': '2024-04-11T06:40:10.218661Z',
 'endTimeUtc': '2024-04-11T06:48:14.442557Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6be10307-4dd1-4e75-8f56-819e866d9b5e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '25',
  'best_child_run_id': 'HD_bac65b68-437e-4e9b-83f8-a42d7476509a_9',
  'score': '0.8',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_bac65b68-437e-4e9b-83f8-a42d7476509a_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'az

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [61]:
import joblib

### YOUR CODE HERE ###
# current primary metric is accuracy. 
best_run = hdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())

{'Regularization Strength:': 5.0, 'Max iterations:': 200, 'Accuracy': 0.8}


In [88]:
#Save the best model
joblib.dump(best_model, filename="./hyperdrive_bestmodel.pkl")


['./hyperdrive_bestmodel.pkl']

In [77]:
#register the best model
best_model = best_run.register_model(model_name='HyperDrive_CapstoneProj_Exp0_BestModel', model_path='.')
best_model

Model(workspace=Workspace.create(name='udc-machinelearning01', subscription_id='7023dccf-8c31-41db-809e-9f442490bfe7', resource_group='rg-udc-ml-dev-eastus-01'), name=HyperDrive_CapstoneProj_Exp0_BestModel, id=HyperDrive_CapstoneProj_Exp0_BestModel:5, version=5, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.



TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

